In [3]:
import tensorflow as tf
import numpy as np

In [4]:
from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [5]:
print("Shapes of Datasets")
print("X_train : ", X_train.shape)
print("X_test : ", X_test.shape)
print("Y_train : ", Y_train.shape)
print("Y_test : ", Y_test.shape)

Shapes of Datasets
X_train :  (60000, 28, 28)
X_test :  (10000, 28, 28)
Y_train :  (60000,)
Y_test :  (10000,)


In [6]:
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session()

In [7]:
y_train_oh = tf.one_hot(Y_train, depth = 10)
y_test_oh = tf.one_hot(Y_test, depth = 10)
Y_train = sess.run(y_train_oh)
Y_test = sess.run(y_test_oh)

### Initializing Weights

In [8]:
input_width = 28
input_height = 28
input_channels =1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2

In [9]:
weights = {
    'wc1' : tf.Variable(tf.random.normal([conv1_k,conv1_k, input_channels, n_conv1])),
    'wc2' : tf.Variable(tf.random.normal([conv2_k,conv2_k, n_conv1, n_conv2])),
     #                                      k,k,channels,units 
    
    'wh1':tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    'wo':tf.Variable(tf.random.normal([n_hidden, n_out]))
}


biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out])),
    
}

In [10]:
tf.compat.v1.trainable_variables()

[<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32>,
 <tf.Variable 'Variable_1:0' shape=(5, 5, 32, 64) dtype=float32>,
 <tf.Variable 'Variable_2:0' shape=(3136, 1024) dtype=float32>,
 <tf.Variable 'Variable_3:0' shape=(1024, 10) dtype=float32>,
 <tf.Variable 'Variable_4:0' shape=(32,) dtype=float32>,
 <tf.Variable 'Variable_5:0' shape=(64,) dtype=float32>,
 <tf.Variable 'Variable_6:0' shape=(1024,) dtype=float32>,
 <tf.Variable 'Variable_7:0' shape=(10,) dtype=float32>]

### Forward Propogation

In [11]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x,weights,padding='SAME', strides = [1,strides, strides,1])
    out = tf.nn.bias_add(out,bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1,k,k,1], strides = [1,k,k,1])

In [12]:
def cnn(x, weights, biases):
    x = tf.reshape(x, shape = [-1,input_height,input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']),biases['bh1']) 
    hidden_output = tf.nn.relu(hidden_output_before_activation)
    
    #output layer (using identity function)
    output = tf.add(tf.matmul(hidden_output, weights['wo']),biases['bo']) 
    return output

In [13]:
x = tf.compat.v1.placeholder("float", [None, input_height, input_width])
y = tf.compat.v1.placeholder(tf.int32, [None, n_out])
pred = cnn(x, weights, biases)

In [14]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels = y))

In [15]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [16]:
sess.run(tf.compat.v1.global_variables_initializer())

### Dropout layer

In [17]:
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape = [-1,input_height,input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob) 
    
    #output layer (using identity function)
    output = tf.add(tf.matmul(hidden_output, weights['wo']),biases['bo']) 
    return output

In [19]:
x = tf.compat.v1.placeholder("float", [None, input_height, input_width])
y = tf.compat.v1.placeholder(tf.int32, [None, n_out])
keep_prob = tf.compat.v1.placeholder("float")
pred = cnn(x, weights, biases,keep_prob)

In [20]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels = y))

In [21]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [22]:
sess.run(tf.compat.v1.global_variables_initializer())

In [23]:
batch_size = 100
for i in range(25):
    num_batches = int(len(X_train)/batch_size)
    total_cost = 0
    i=0
    j=100
    for p in range(num_batches):
        batch_X, batch_Y = X_train[i:j], Y_train[i:j]
        i+=batch_size
        j+=batch_size
        c, _ = sess.run([cost,optimize], feed_dict={x:batch_X , y:batch_Y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

KeyboardInterrupt: 

In [ ]:
predictions = tf.argmax(pred,1)
correct_labels = tf.argmax(y,1)
correct_predictions = tf.equal(predictions,correct_labels)

predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict = {x: X_train, y: Y_train, keep_prob : 1.0})
correct_preds.sum()